# Amazon Bedrock, Kendra, OpenSearch using Langchain

In these notebooks, you will use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundational Models.


## Prerequisites
### ⚠️ Please make sure to use this notebook in an account and Region with Bedrock enable
### ⚠️ These notebook require the **`Data Science 3.0`** kernel in Amazon SageMaker Studio.
### ⚠️ If you plan to use a third-party foundation model from Anthropic, AI21, etc, you will need to allow-list the account for the 3rd party model provider.  This may require a few days, so please plan ahead.
### ⚠️ Some of the samples may be hard-coded to a given region. ***Please do not can change the region.***
### ⚠️ Be sure the role used by SageMaker has the policies for ***Full Access*** to Bedrock and Kendra. 
### ⚠️ We will use a knowledge base created in Bedrock Console to compare the results with Kendra Index

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.

#### Use a different AWS Region

If you're running this notebook from your own computer or a SageMaker notebook in a different AWS Region from where Bedrock is set up, you can un-comment the `os.environ['AWS_DEFAULT_REGION']` line below and specify the region to use.

#### Use a specific profile

In case you're running this notebook from your own computer where you have setup the AWS CLI with multiple profiles, and the profile which has access to Bedrock is not the default one, you can un-comment the `os.environ['AWS_PROFILE']` line below and specify the profile to use.

#### Use a different role

In case you or your company has setup a specific, separate [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) to access Bedrock, you can specify it by un-commenting the `os.environ['BEDROCK_ASSUME_ROLE']` line below. Ensure that your current user or role have permissions to [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) such role.

#### Use a custom service endpoint URL

If you need to use a custom [service endpoint URL](https://docs.aws.amazon.com/general/latest/gr/rande.html) to access Bedrock as part of the preview, you'll need to un-comment and edit the `os.environ['BEDROCK_ENDPOINT_URL']` line below.

### Please follow the next lines to download, unzip and install the libraries required

In [40]:
!pip install -q -r ../requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.40 requires botocore==1.34.40, but you have botocore 1.34.37 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Let's start by listing the different models supported by bedrock on this account. If you want to use a model that is not listed here, please see the Bedrock documentation on how to get access to more models. 

In [41]:
import boto3
import json
import os 

region = os.environ.get("AWS_REGION")

bedrock = boto3.client('bedrock', region, endpoint_url='https://bedrock.'+region+'.amazonaws.com')
response = bedrock.list_foundation_models()

# Extract modelIds from the response data
model_ids = [model["modelId"] for model in response.get('modelSummaries')]

# Print the list of modelIds
for model_id in model_ids:
    print(model_id)


amazon.titan-tg1-large
amazon.titan-image-generator-v1:0
amazon.titan-image-generator-v1
amazon.titan-embed-g1-text-02
amazon.titan-text-lite-v1:0:4k
amazon.titan-text-lite-v1
amazon.titan-text-express-v1:0:8k
amazon.titan-text-express-v1
amazon.titan-embed-text-v1:2:8k
amazon.titan-embed-text-v1
amazon.titan-embed-image-v1:0
amazon.titan-embed-image-v1
stability.stable-diffusion-xl
stability.stable-diffusion-xl-v0
stability.stable-diffusion-xl-v1:0
stability.stable-diffusion-xl-v1
ai21.j2-grande-instruct
ai21.j2-jumbo-instruct
ai21.j2-mid
ai21.j2-mid-v1
ai21.j2-ultra
ai21.j2-ultra-v1
anthropic.claude-instant-v1:2:100k
anthropic.claude-instant-v1
anthropic.claude-v1
anthropic.claude-v2:0:18k
anthropic.claude-v2:0:100k
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-v2:1
anthropic.claude-v2
cohere.command-text-v14:7:4k
cohere.command-text-v14
cohere.command-light-text-v14:7:4k
cohere.command-light-text-v14
cohere.embed-english-v3
cohere.embed-multilingual-v3
meta.l

### We need to setup some variables like bedrock end point, region where you have bedrock and the AWS profiles used. 

In [42]:
BWB_ENDPOINT_URL = 'https://bedrock-runtime.'+region+'.amazonaws.com'
BWB_PROFILE_NAME = 'bedrock-runtime'
BWB_REGION_NAME = region    

# let's use 'store' so you can use these variables in other notebooks
%store BWB_ENDPOINT_URL
%store BWB_PROFILE_NAME
%store BWB_REGION_NAME

#%store -r BWB_ENDPOINT_URL
#%store -r BWB_PROFILE_NAME
#%store -r BWB_REGION_NAME

print(BWB_ENDPOINT_URL)
print(BWB_PROFILE_NAME)
print(BWB_REGION_NAME)

Stored 'BWB_ENDPOINT_URL' (str)
Stored 'BWB_PROFILE_NAME' (str)
Stored 'BWB_REGION_NAME' (str)
https://bedrock-runtime.us-east-1.amazonaws.com
bedrock-runtime
us-east-1


### Next, let's create a bedrock client using boto3. We will run a basic question using titan model. If the response is Manchester, this means we were able to setup bedrock client and session properly!!

In [43]:
import os
import json
import boto3

#This creates a Bedrock client.
session = boto3.Session() #sets the profile name to use for AWS credentials

bedrock = session.client(
    service_name='bedrock-runtime', #creates a Bedrock client for invoking models 
    region_name=BWB_REGION_NAME,
    endpoint_url=BWB_ENDPOINT_URL
) 

#Here we are identifying the model to use, the prompt, and the inference parameters for the specified model.
bedrock_model_id = "anthropic.claude-v2" #set the model to Claude
prompt = """
Human: What is the largest city in New Hampshire?
Assistant:
""" 
#the prompt to send to the model
#body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 512, "stopSequences": [], "temperature": 0, "topP": 0.9 } } ) #build the request payload
body = json.dumps({"prompt": prompt,"max_tokens_to_sample": 512,"stop_sequences":[],"temperature":0,"top_p":0.9})

#We use Bedrock's invoke_model function to make the call.
response = bedrock.invoke_model(body=body, modelId=bedrock_model_id, accept='application/json', contentType='application/json') #send the payload to Bedrock

#This extracts & prints the returned text from the model's response JSON.
response_body = json.loads(response.get('body').read()) # read the response
print(response_body)




{'completion': ' The largest city in New Hampshire is Manchester.', 'stop_reason': 'stop_sequence', 'stop': '\n\nHuman:'}


### Intro to LangChain

we will show how to make a basic call to Bedrock using LangChain. While similar in functionality to the previous cells where we called Bedrock through Boto3, in this section we will use LangChain so you can compare the two approaches. LangChain can abstract away many of the details of using the Boto3 client, especially when you want to focus on text in and text out. If you need the full control enabled by the Boto3 client, it will always be an option for you. Boto3 may require more code, but it will give you full access to the JSON request and response objects.

You can build the application code by copying the code snippets below and pasting into the indicated Python file.

#### If "anthropic.claude-v2" is not available, please use:  "amazon.titan-tg1-large" 

In [44]:
import os
from langchain.llms.bedrock import Bedrock


#This creates a Bedrock client. The client allows us to make standard LangChain calls and have them adapted automatically to work with Bedrock's models.
llm = Bedrock( #create a Bedrock llm client
    region_name=BWB_REGION_NAME, #sets the region name (if not the default)
    endpoint_url=BWB_ENDPOINT_URL, #sets the endpoint URL (if necessary)
    model_id="anthropic.claude-v2", model_kwargs={'max_tokens_to_sample':200} #if anthropic is not available use:  "amazon.titan-tg1-large"
)

#With LangChain, the prompt is the minimum payload we need to send to Bedrock. We'll also learn how to set inference parameters in a later lab.
prompt = "What is the largest city in Vermont?"

#the prompt to send to the model
body = json.dumps({"prompt": prompt,"max_tokens_to_sample": 512,"stop_sequences":[],"temperature":0,"top_p":0.9})

#Call Bedrock API
response_text = llm.invoke(prompt) #return a response to the prompt
print(response_text)



 Burlington is the largest city in Vermont.


### Using Kendra and LangChain
We let's run get ready for using Kendra and Langchain in our example

In [45]:
from langchain.retrievers import AmazonKendraRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.chains.llm import LLMChain
import sys

### Life is not black and white. These lines will add some colors to Bedrock's output

In [46]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
MAX_HISTORY_LENGTH = 5

### As final Step we will use Kendra as retriever and the use it within langchain to extend the answer based on the documents store in Kendra
## We will follow these steps:
1. Define a retriever using the kendra index
   - NOTE: ⚠️ Remember to update the kendra Index ID
   
2. Define a prompt template
3. Create a conversation chain

In [47]:
def build_chain():
  region = BWB_REGION_NAME 
  kendra_index_id = '35c6def6-28b7-4214-8b39-ed40e27ebf8b'#this is the "KENDRA_INDEX_ID", make sure to get it from Kendra and that the data source is synced up
  credentials_profile_name = 'BedrockUserRole'
  
  retriever = AmazonKendraRetriever(index_id=kendra_index_id,top_k=5,region_name=region)

  prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  {context}

  Question: {question}
  Assistant:"""
  PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )
  
  qa = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        retriever=retriever, 
        return_source_documents=True, #This parameter will show the list of url or links used in the answer provided. Possible values True and False
        combine_docs_chain_kwargs={"prompt":PROMPT},
        verbose=False) #This parameter will show you the sections of the documents used to create the answer. Possible values True and False

  return qa
def run_chain(chain, prompt: str, history=[]):
  return chain({"question": prompt, "chat_history": history})


### You can change the query asked to the LLM, these are some options:
    
    - How old is Catherine J. Randall?
    - How old is Phillip W. Webb?
    - What is Angus R. Cooper, III's current position and what is the name of the organization he/she currently works for?
    - What year did Robert D. Powers joined the board of directors?
    - What committees is Catherine J. Randall a member of?
    - What is Phillip W. Webb's current position and what is the name of the organization he/she currently works for?

In [48]:
chat_history = []
qa = build_chain()
query = 'who is older between Powers and Randall?'  #Feel free to change this question
chat_history = []
if (len(chat_history) == MAX_HISTORY_LENGTH):
  chat_history.pop(0)
result = run_chain(qa, query, chat_history)
chat_history.append((query, result["answer"]))
print(">", query, end=" ", flush=True)  
print(bcolors.OKGREEN + result['answer'] + bcolors.ENDC)
if 'source_documents' in result:
  print(bcolors.OKGREEN + 'Sources:')
  for d in result['source_documents']:
    print(d.metadata['source'])

> who is older between Powers and Randall?  Based on the document excerpts, Robert D. Powers is 69 years old and Catherine J. Randall is 69 years old. Therefore, Powers and Randall are the same age.
Sources:
https://s3.us-east-1.amazonaws.com/genaiconf-2958e600/data/14A/0000003153-20-000004.html
https://s3.us-east-1.amazonaws.com/genaiconf-2958e600/data/14A/0000003153-20-000004.html
https://s3.us-east-1.amazonaws.com/genaiconf-2958e600/data/14A/0000003153-20-000004.html
https://s3.us-east-1.amazonaws.com/genaiconf-2958e600/data/14A/0000003153-20-000004.html
https://s3.us-east-1.amazonaws.com/genaiconf-2958e600/data/14A/0000003153-20-000004.html


# Now, let's change the Knowledge based, let's use the one we created in the Bedrock console and compare the results. 


## ⚠️Warning: Remember to sync up the Knowledge base in Bedrock console and copy the Knowledge Base ID, See the KB Menu in Bedrock's Console
### Syncing will take about 2 minutes 

### Remember: The following Knowledge base is using Amazon OpenSearch Serverless and the embedding selected during the creation of the KB. The result could be different. Those can be improve or fine-tuned by changing several parameter such as: Chunking, Embedding model and others

### In the following cell, we will be testing the knowledge base only, you will see no Text2Text Bedrock Model is associated.

In [49]:
# Please update the knowledge base Id here :) 
KBaseId = "AUYJTRQHQ8"

In [50]:
import boto3
import json
client = boto3.client('bedrock-agent-runtime')

response = client.retrieve(
    knowledgeBaseId=KBaseId,
    retrievalQuery={
        'text': 'who is the oldest person in the board?'
    },
    retrievalConfiguration={
        'vectorSearchConfiguration': {
            'numberOfResults': 1
        }
    },
    nextToken='string'
)

# Get retrievalResults 
results = response['retrievalResults']

# Process each result
for result in results:
  text = result['content']['text']
  
  # Replace \t with spaces
  text = text.replace('\t', ' ') 
  print(text)

O.B. Grayson Hall, Jr. - Director since 2015 Mr. Hall, 62, served as Executive Chairman of Regions Financial Corporation from July 2018 until his retirement in December 2018. He previously served as Chairman of Regions Financial Corporation from May 2013 through July 2018 and Chief Executive Officer from April 2010 through July 2018. He served as President of Regions Financial Corporation from 2009 through December 2017. Mr. Hall serves on the Boards of Directors of Vulcan Materials Company and Great Southern Wood Holdings, Inc. He previously served as a representative on the Federal Advisory Council of the Federal Reserve Bank from 2014 to 2016 and on the Board of the Federal Reserve Bank of Atlanta from 2017 to 2018. Mr. Hall also serves on the Boards of numerous civic organizations, including the Newcomen Society of Alabama and the National Christian Foundation of Alabama and is a trustee of the Crimson Tide Foundation. Mr. Hall's experience in the business community, as well as his

### Now we will use the Text Generation from our Bedrock Model to consolidate the answer we got in the previous cell

In [51]:
response = client.retrieve_and_generate(
    input={
        'text': 'who is the oldest person of the board?'
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': KBaseId,
            'modelArn': 'arn:aws:bedrock:'+BWB_REGION_NAME+'::foundation-model/anthropic.claude-v2'
        }
    }
)

text = response["output"]["text"]

print(text)

retrieved_references = response['citations'][0]['retrievedReferences']
print("These are the citations from the documents collected in Amazon OpenSearch Serverless:")
for ref in retrieved_references:
    print(ref['location']['s3Location']['uri'])
          #['content']['text']+"/n")


Based on the search results, R. Mitchell Shackleford III, who is 68 years old, appears to be the oldest member of the board.
These are the citations from the documents collected in Amazon OpenSearch Serverless:
s3://genaiconf-546bcdd0/data/14A_frags/0000003153-20-000004.shackleford.txt
s3://genaiconf-546bcdd0/data/14A_frags/0000003153-20-000004.shackleford.pdf


### Finally, we will change the question a bit to confirm the information given by our Model is accurate or not, we can also ask the same question when using Kendra Index and check for differences

In [52]:
import sagemaker

response = client.retrieve_and_generate(
    input={
        'text': 'is Dr. Randall the oldest member of the board?'
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': KBaseId,
            'modelArn': 'arn:aws:bedrock:'+BWB_REGION_NAME+'::foundation-model/anthropic.claude-v2'
        }
    }
)
text = response["output"]["text"]

print(text)
retrieved_references = response['citations'][0]['retrievedReferences']
print("These are the citations from the documents collected in Amazon OpenSearch Serverless:")
for ref in retrieved_references:
    print(ref['location']['s3Location']['uri'])
          #['content']['text']+"/n")


Yes, Dr. Randall is the oldest member of the board at 69 years old.
These are the citations from the documents collected in Amazon OpenSearch Serverless:
s3://genaiconf-546bcdd0/data/14A_frags/0000003153-20-000004.randall.pdf
s3://genaiconf-546bcdd0/data/14A_frags/0000003153-20-000004.randall.txt
